In [2]:
!pip install langchain
!pip install huggingface-hub
!pip install faiss-cpu
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 5.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [4]:
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.4 MB/s eta 0:00:00


In [25]:
from langchain.document_loaders import TextLoader
loader = TextLoader('/content/vinland-saga.txt')
document = loader.load()
print(document)

[Document(page_content="Vinland Saga (Japanese: ヴィンランド・サガ, Hepburn: Vinrando Saga) is a Japanese historical manga series written and illustrated by Makoto Yukimura. The series is published by Kodansha, and was first serialized in the boys-targeted manga magazine Weekly Shōnen Magazine before moving to the monthly manga magazine Monthly Afternoon, aimed at young adult men. As of June 2023, its chapters have been collected in 27 tankōbon volumes. Vinland Saga has been licensed for English-language publication by Kodansha USA. The story is a dramatization of the story of Thorfinn Karlsefni and his expedition to find Vinland, with the majority of the story covering his fictional counterpart's transition from a bloodthirsty, revenge-filled teenager into a pacifistic young man; juxtaposed against this is the rise to power of King Canute, the journey of his own counterpart directly contrasting with that of Thorfinn's.\n\nInspired by the King of Norway Olaf Tryggvason, Yukimura decided to writ

# Preprocessing

In [6]:
import textwrap

In [26]:
def wrap_text_preserve_newlines(text, width=110):
  # split input text based on new line (\n) character
  lines = text.split('\n')

  # wrap each line individually
  wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

  # join the wrapped lines back together using newline character
  wrapped_text = '\n'.join(wrapped_lines)

  return wrapped_text

print(wrap_text_preserve_newlines(str(document[0])))

page_content="Vinland Saga (Japanese: ヴィンランド・サガ, Hepburn: Vinrando Saga) is a Japanese historical manga series
written and illustrated by Makoto Yukimura. The series is published by Kodansha, and was first serialized in
the boys-targeted manga magazine Weekly Shōnen Magazine before moving to the monthly manga magazine Monthly
Afternoon, aimed at young adult men. As of June 2023, its chapters have been collected in 27 tankōbon volumes.
Vinland Saga has been licensed for English-language publication by Kodansha USA. The story is a dramatization
of the story of Thorfinn Karlsefni and his expedition to find Vinland, with the majority of the story covering
his fictional counterpart's transition from a bloodthirsty, revenge-filled teenager into a pacifistic young
man; juxtaposed against this is the rise to power of King Canute, the journey of his own counterpart directly
contrasting with that of Thorfinn's.\n\nInspired by the King of Norway Olaf Tryggvason, Yukimura decided to
write Vinland 

# Text Splitting

In [27]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
docs = text_splitter.split_documents(document)

print(docs[0])
print(len(docs))

page_content="Vinland Saga (Japanese: ヴィンランド・サガ, Hepburn: Vinrando Saga) is a Japanese historical manga series written and illustrated by Makoto Yukimura. The series is published by Kodansha, and was first serialized in the boys-targeted manga magazine Weekly Shōnen Magazine before moving to the monthly manga magazine Monthly Afternoon, aimed at young adult men. As of June 2023, its chapters have been collected in 27 tankōbon volumes. Vinland Saga has been licensed for English-language publication by Kodansha USA. The story is a dramatization of the story of Thorfinn Karlsefni and his expedition to find Vinland, with the majority of the story covering his fictional counterpart's transition from a bloodthirsty, revenge-filled teenager into a pacifistic young man; juxtaposed against this is the rise to power of King Canute, the journey of his own counterpart directly contrasting with that of Thorfinn's." metadata={'source': '/content/vinland-saga.txt'}
3


# Embedding

In [11]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YOUR_API_KEY"

In [28]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
embeddings = HuggingFaceEmbeddings()
db = FAISS.from_documents(docs, embeddings)

query = "Who is animating the second season of Vinland Saga?"

doc = db.similarity_search(query)

print(wrap_text_preserve_newlines(str(doc[0].page_content)))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


By August 2022, Vinland Saga had over 7 million copies in circulation. The series won the Grand Prize of the
13th Japan Media Arts Festival in 2009 and the 36th Kodansha Manga Award for Best General Manga in 2012. A
24-episode anime television series adaptation, animated by Wit Studio, aired on NHK General TV from July to
December 2019. A second season, animated by MAPPA, premiered on Tokyo MX and BS11 in January 2023. The manga
has been critically acclaimed with praise given to the story, art, characters, themes and setting.


# Q-A

In [29]:
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub

llm = HuggingFaceHub(repo_id='google/flan-t5-xxl', model_kwargs={"temperature":0.8, "max-length":512})

chain = load_qa_chain(llm, chain_type="stuff")

In [33]:
queryText = "Who is the author of Vinland Saga?"
docsResult = db.similarity_search(queryText)
print(chain.run(input_documents = docsResult, question = queryText))

Makoto Yukimura


In [34]:
queryText = "What Vinland Saga is about?"
docsResult = db.similarity_search(queryText)
print(chain.run(input_documents = docsResult, question = queryText))

The story is a dramatization of the story of Thorfinn Karlsefni and his expedition to find Vinland, with the majority of the story covering his fictional counterpart's transition from a bloodthirsty, revenge-filled teenager into a pacifistic young man; juxtaposed against this is the rise to power of King Canute, the journey of his own counterpart directly contrasting with that of Thorfinn's


In [35]:
queryText = "Why is Thorfinn want to find Vinland?"
docsResult = db.similarity_search(queryText)
print(chain.run(input_documents = docsResult, question = queryText))

in order to make a land where people from different races are able to live together peacefully
